In [236]:
import numpy as np
import networkx as nx
import itertools
import pandas as pd
from scipy.spatial import distance

from pgmpy.models import BayesianNetwork as BN
from pgmpy.factors.discrete import TabularCPD as cpd
from pgmpy.inference import VariableElimination

import torch
import torch.nn as nn

In [237]:
# check if pgmpy using the same seed
np.random.seed(1985)

n_samples = 10**4

In [238]:
M0 = BN([('Battery_Size','Weight'),('Battery_Size','Range'),('Weight','Efficiency'),('Efficiency','Range')])


cpdB = cpd(variable='Battery_Size',
          variable_card=4,
          values=[[0.3],[0.35],[0.15],[0.2]],
          evidence=None,
          evidence_card=None)
cpdS = cpd(variable='Weight',
          variable_card=3,
          values=[[0.6,0.5,0.3,0.1],[0.3,0.45,0.55,0.2],[0.1,0.05,0.15,0.7]],
          evidence=['Battery_Size'],
          evidence_card=[4])
cpdT = cpd(variable='Efficiency',
          variable_card=4,
          values=[[0.5,0.3,0.1],[0.3,0.3,0.15],[0.1,0.3,0.15],[0.1,0.1,0.6]],
          evidence=['Weight'],
          evidence_card=[3])
cpdC = cpd(variable='Range',
          variable_card=4,
          values = [
              [0.5,0.4,0.3,0.1,0.55,0.43,0.35,0.1,0.65,0.5,0.2,0.15,0.7,0.55,0.35,0.18],
              [0.35,0.35,0.35,0.15,0.35,0.32,0.4,0.2,0.3,0.35,0.45,0.25,0.25,0.35,0.3,0.22],
              [0.10,0.15,0.2,0.35,0.08,0.15,0.2,0.4,0.04,0.1,0.25,0.35,0.04,0.07,0.2,0.4],
              [0.05,0.1,0.15,0.4,0.02,0.1,0.05,0.3,0.01,0.05,0.1,0.25,0.01,0.03,0.15,0.2]
            ],
          evidence=['Efficiency','Battery_Size'],
          evidence_card=[4,4])

M0.add_cpds(cpdB,cpdS,cpdT,cpdC)
M0.check_model()

True

In [239]:
M1 = BN([('Weight_','Efficiency_'),('Efficiency_','Range_')])

cpdS = cpd(variable='Weight_',
          variable_card=3,
          values=[[0.2],[0.2],[0.6]],
          evidence=None,
          evidence_card=None)
cpdT = cpd(variable='Efficiency_',
          variable_card=4,
          values=[[0.5,0.3,0.1],[0.3,0.3,0.15],[0.1,0.3,0.15],[0.1,0.1,0.6]],
          evidence=['Weight_'],
          evidence_card=[3])
cpdC = cpd(variable='Range_',
          variable_card=4,
          values=[[0.05,0.15,0.1,0.25],[0.15,0.15,0.25,0.45],[0.3,0.35,0.4,0.25],[0.5,0.35,0.25,0.05]],
          evidence=['Efficiency_'],
          evidence_card=[4])

M1.add_cpds(cpdS,cpdT,cpdC)
M1.check_model()

True

In [240]:
def inverse_fx(f,x):
    return list(np.array(list(f.keys()))[np.where(np.in1d(np.array(list(f.values())),x))[0]])


def mapping(array, idx):
    res = np.zeros_like(array)
    for i in range(len(res[0])):
        res[idx[i]][i] = 1
    return res   


def create_alphas(F,alpha_keys,cards_domain,cards_codomain):
    alpha = {}
    for k in alpha_keys:
        domain = inverse_fx(F,k)
        card_domain = 1
        for d in domain:
            card_domain = card_domain * cards_domain(d)
        card_codomain = cards_codomain(k)
        dim = np.random.rand(card_codomain,card_domain)
        dim = dim/dim.sum(axis=0)[None,:]
        idx = np.argmax(dim,axis=0)
        dim = mapping(dim,idx)
        alpha[k] = dim
    return alpha    

In [241]:
R = ['Weight','Efficiency','Range','Battery_Size']

a = {'Battery_Size': 'Weight_',
     'Weight': 'Weight_',
     'Efficiency': 'Efficiency_',
     'Range': 'Range_'}

alpha_keys = ['Weight_','Efficiency_','Range_']
alphas = create_alphas(a, alpha_keys, M0.get_cardinality, M1.get_cardinality)
alphas

{'Weight_': array([[0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1.],
        [1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.]]),
 'Efficiency_': array([[0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [1., 0., 1., 0.],
        [0., 0., 0., 0.]]),
 'Range_': array([[1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 0.],
        [0., 1., 1., 0.]])}

In [242]:
from src.SCMMappings import Abstraction
A = Abstraction(M0,M1,R,a,alphas)

In [243]:
err = A.evaluate_abstraction_error(verbose=True)

- Checking ['Weight_'] -> ['Efficiency_']: True
- Checking ['Weight_'] -> ['Range_']: True
- Checking ['Weight_'] -> ['Efficiency_', 'Range_']: True
- Checking ['Efficiency_'] -> ['Weight_']: False
---- Checking ['Efficiency'] -> ['Battery_Size', 'Weight']: False
- Checking ['Efficiency_'] -> ['Range_']: True
- Checking ['Efficiency_'] -> ['Weight_', 'Range_']: True
- Checking ['Range_'] -> ['Weight_']: False
---- Checking ['Range'] -> ['Battery_Size', 'Weight']: False
- Checking ['Range_'] -> ['Efficiency_']: False
---- Checking ['Range'] -> ['Efficiency']: False
- Checking ['Range_'] -> ['Weight_', 'Efficiency_']: False
---- Checking ['Range'] -> ['Battery_Size', 'Weight', 'Efficiency']: False
- Checking ['Weight_', 'Efficiency_'] -> ['Range_']: True
- Checking ['Weight_', 'Range_'] -> ['Efficiency_']: True
- Checking ['Efficiency_', 'Range_'] -> ['Weight_']: False
---- Checking ['Efficiency', 'Range'] -> ['Battery_Size', 'Weight']: False

 7 legitimate pairs of sets out of 49 possbi

In [244]:
def one_hot_vector(mat2d, dims):
    res = np.zeros((mat2d.shape[0],dims[0] * dims[1]))
    for i in range(mat2d.shape[0]):
        tmp = np.zeros((dims[0],dims[1]))
        tmp[mat2d[i][0],mat2d[i][1]] = 1
        vec = tmp.flatten()
        res[i] = vec
    return res

def one_hot_encoding(data, num_classes):
    res = np.zeros((data.shape[0],num_classes))
    for i in range(data.shape[0]):
        tmp = np.zeros((num_classes))
        tmp[data[i]] = 1
        res[i] = tmp
    return res 


def normalise(mat2d):
    for i in range(mat2d.shape[0]):
        tmp = mat2d[i]
        tmp /= tmp.sum()
        mat2d[i] = tmp
    return mat2d    
    

In [245]:
# Data Collection
n_samples = 10**3

# Base model
M0do = M0.do('Battery_Size','Weight')
lowlevel_data = M0do.simulate(n_samples=n_samples, show_progress=False)
con_lowlevel_data = np.concatenate((np.expand_dims(lowlevel_data['Battery_Size'].to_numpy(dtype=np.float32),axis=1), np.expand_dims(lowlevel_data['Weight'].to_numpy(dtype=np.float32),axis=1)), axis=1)
hot_lowlevel_data = one_hot_vector(con_lowlevel_data.astype(int),(4,3))

# Abstract model
M1do = M1.do('Weight_')
high_level_samples = [M1do.simulate(n_samples=1, evidence={'Weight_': np.where(alphas['Weight_'][:,lowlevel_data.loc[i]['Battery_Size']]==1)[0][0], 'Weight_': np.where(alphas['Weight_'][:,lowlevel_data.loc[i]['Weight']]==1)[0][0]}, show_progress=False) for i in range(lowlevel_data.shape[0])]
highlevel_data = pd.concat(high_level_samples)
exp_highlevel_data = np.expand_dims(highlevel_data['Weight_'].to_numpy(dtype=np.float32),axis=1)
hot_highlevel_data = one_hot_encoding(exp_highlevel_data.astype(int),3)

# Target low
target_low = np.expand_dims(lowlevel_data['Range'].to_numpy(dtype=np.float32),axis=1)
hot_target_low = one_hot_encoding(target_low.astype(int),4)

# Target high
target_high = np.expand_dims(highlevel_data['Range_'].to_numpy(dtype=np.float32),axis=1)
hot_target_high = one_hot_encoding(target_high.astype(int),4)

In [246]:
lr = 0.005
hiddensize = 4
num_epochs = 500

In [247]:
# Mechanisms
# M1 mechanism shape: (4, 3)
# M0 mechanism shape: (4, 12)
# make mechanism with no noise
discrete_mode = True

inferM0 = VariableElimination(M0)
M0_joint_AD = inferM0.query(['Battery_Size','Weight','Range'],show_progress=False)
M0_joint_A = inferM0.query(['Battery_Size','Weight'],show_progress=False)
M0_cond_DA = M0_joint_AD / M0_joint_A

dims = M0_cond_DA.values.shape
n = dims[2]
mech_low = np.zeros((dims[2],dims[0] * dims[1]))
for i in range(n):
    tmp = M0_cond_DA.values[:,:,i]
    vec = tmp.flatten()
    mech_low[i] = vec

inferM1 = VariableElimination(M1)
M1_joint_XZ = inferM1.query(['Weight_','Range_'],show_progress=False)
M1_joint_X = inferM1.query(['Weight_'],show_progress=False)
M1_cond_ZX = M1_joint_XZ / M1_joint_X
mech_high = M1_cond_ZX.values.T

if discrete_mode:
    mech_low = mapping(mech_low,np.argmax(mech_low,axis=0))
    mech_high = mapping(mech_high,np.argmax(mech_low,axis=0))


In [248]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        
        # model 0
        self.fc3 = nn.Linear(12,3)
        self.absS = nn.Softmax()
        
        # model 1        
        self.fc7 = nn.Linear(4,4)
        self.absC = nn.Softmax()
        
       
    def forward(self, input_lowS, mech_low, mech_high):
        # abs_low
        out = self.fc3(input_lowS)
        out = out/0.1
        abs_low = self.absS(out)
        
        # abs_low x mech_high
        abs_mech_high = torch.matmul(abs_low.float(),mech_high.T.float())
        
        # mech_low x abs
        out = self.fc7(mech_low)
        mech_low_abs = self.absC(out)
        
        return abs_low, abs_mech_high, mech_low_abs

In [249]:
model = NeuralNet()

In [250]:
# JSD Loss Function
class JSD_Torch(nn.Module):
    def __init__(self):
        super(JSD_Torch, self).__init__()
    
    def my_kl_div(self,p, q): 
        return torch.sum(p*torch.log((p/q)+1e-7),0)
    
    def forward(self, p, q):
        m = (1./2.)*(p + q)
        return torch.max(torch.sqrt((1./2.)*self.my_kl_div(p,m)+(1./2.)*self.my_kl_div(q,m)))

In [251]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

loss_commutativity = JSD_Torch()

In [252]:
for epoch in range(num_epochs):
    input_lowS = torch.from_numpy(hot_lowlevel_data).float()
    input_mech_low = torch.from_numpy(np.dot(hot_lowlevel_data, mech_low.T)).float()
    input_mech_high = torch.from_numpy(mech_high)
    
    abs_low, abs_mech_high, mech_low_abs  = model(input_lowS, input_mech_low, input_mech_high)
        
    loss = loss_commutativity(abs_mech_high.T,mech_low_abs.T)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch+1) % 5 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

/var/folders/n0/yr8s1fh94295209wh5ynccp40000gn/T/ipykernel_2764/91320153.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  abs_low = self.absS(out)
/var/folders/n0/yr8s1fh94295209wh5ynccp40000gn/T/ipykernel_2764/91320153.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  mech_low_abs = self.absC(out)


Epoch [5/500], Loss: 0.6703
Epoch [10/500], Loss: 0.6599
Epoch [15/500], Loss: 0.6489
Epoch [20/500], Loss: 0.6374
Epoch [25/500], Loss: 0.6253
Epoch [30/500], Loss: 0.6126
Epoch [35/500], Loss: 0.5994
Epoch [40/500], Loss: 0.5855
Epoch [45/500], Loss: 0.5711
Epoch [50/500], Loss: 0.5562
Epoch [55/500], Loss: 0.5408
Epoch [60/500], Loss: 0.5250
Epoch [65/500], Loss: 0.5088
Epoch [70/500], Loss: 0.4925
Epoch [75/500], Loss: 0.4759
Epoch [80/500], Loss: 0.4593
Epoch [85/500], Loss: 0.4428
Epoch [90/500], Loss: 0.4263
Epoch [95/500], Loss: 0.4101
Epoch [100/500], Loss: 0.3946
Epoch [105/500], Loss: 0.3807
Epoch [110/500], Loss: 0.3670
Epoch [115/500], Loss: 0.3540
Epoch [120/500], Loss: 0.3415
Epoch [125/500], Loss: 0.3294
Epoch [130/500], Loss: 0.3182
Epoch [135/500], Loss: 0.3073
Epoch [140/500], Loss: 0.2965
Epoch [145/500], Loss: 0.2865
Epoch [150/500], Loss: 0.2768
Epoch [155/500], Loss: 0.2674
Epoch [160/500], Loss: 0.2588
Epoch [165/500], Loss: 0.2501
Epoch [170/500], Loss: 0.2423


In [253]:
def kl_divergence(p, q): 
    return np.sum(p*np.log((p/q)+1e-7),0)

def js_divergence(p, q):
    m = (1./2.)*(p + q)
    return np.sqrt((1./2.)*kl_divergence(p, m) + (1./2.)*kl_divergence(q, m))

In [254]:
abs_low_acc = 0

# Test data samples
n_test = 100
M0do = M0.do('Battery_Size','Weight')
test_input = M0do.simulate(n_samples=n_test, show_progress=False)
test_input = np.concatenate((np.expand_dims(test_input['Battery_Size'].to_numpy(dtype=np.float32),axis=1), np.expand_dims(test_input['Weight'].to_numpy(dtype=np.float32),axis=1)), axis=1)
test_input = one_hot_vector(test_input.astype(int),(4,3))

# Network inputs:
test_input_torch = torch.from_numpy(test_input)
test_mech_high = torch.from_numpy(mech_high)

for i in range(n_test):
    test_mech_low = torch.from_numpy(np.dot(test_input_torch[i], mech_low.T))
    abs_low, abs_mech_high, mech_low_abs  = model(test_input_torch[i].float(), test_mech_low.float(), test_mech_high)
    
    # ground truth upper
    upper_path = np.dot(test_input[i],mech_low.T)
    upper_path = np.dot(upper_path,alphas['Range_'])
    # ground truth lower
    lower = np.dot(test_input[i], alphas['Weight_'].T)
    lower_path = np.dot(lower,mech_high.T)
    # ground truth abstract
    alpha_idx = np.where(test_input[i])
    alpha_idx = alpha_idx[0].item()
    abs_low_truth = alphas["Weight_"][:,alpha_idx]
    
    if np.argmax(abs_low.detach().numpy()) == np.argmax(abs_low_truth):
        abs_low_acc += 1

    print('-------------------------\n')
    print(f"Given I perform the intervention A={test_input[i]}")
    
    print('\nThe upper path produces:')
    print(f'network: {np.around(mech_low_abs.detach().numpy(),5)} <-> {np.around(upper_path,5)} ground truth')
    p_upper = np.around(mech_low_abs.detach().numpy(),5)
    q_upper = np.around(upper_path,5)
    jsd_upper = js_divergence(p_upper,q_upper)
    print(f'JSD between network and ground truth upper path: {jsd_upper}')    
    
    print('\nThe lower path produces:')
    print(f'network: {np.around(abs_mech_high.detach().numpy(),5)} <-> {np.around(lower_path,5)} ground truth')
    p_lower = np.around(abs_mech_high.detach().numpy(),5)
    q_lower = np.around(lower_path,5)
    jsd_lower = js_divergence(p_lower,q_lower)
    print(f'JSD between network and ground truth upper path: {jsd_lower}')
    
    print('\nThe abstraction path produces:')
    print(f'network: {np.around(abs_low.detach().numpy(),5)} <-> {abs_low_truth} ground truth')
    p_abs_low = np.around(abs_low.detach().numpy(),5)
    q_abs_low = abs_low_truth
    jsd_abs_low = js_divergence(p_abs_low,q_abs_low)
    print(f'JSD between network and ground truth upper path: {jsd_abs_low}')
    
    print('\nNetwork JSD:')
    jsd_network = js_divergence(abs_mech_high.detach().numpy(),mech_low_abs.detach().numpy())
    print(f'JSD between upper and lower path in the network: {jsd_network}')
    
    print('\nGround truth JSD:')
    jsd_truth = js_divergence(lower_path,upper_path)
    print(f'JSD between upper and lower path in the ground truth: {jsd_truth}')
    
    print('\n-------------------------')


/var/folders/n0/yr8s1fh94295209wh5ynccp40000gn/T/ipykernel_2764/91320153.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  abs_low = self.absS(out)
/var/folders/n0/yr8s1fh94295209wh5ynccp40000gn/T/ipykernel_2764/91320153.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  mech_low_abs = self.absC(out)
/var/folders/n0/yr8s1fh94295209wh5ynccp40000gn/T/ipykernel_2764/2500776270.py:2: RuntimeWarning: invalid value encountered in true_divide
  return np.sum(p*np.log((p/q)+1e-7),0)


-------------------------

Given I perform the intervention A=[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

The upper path produces:
network: [0.99  0.002 0.003 0.006] <-> [1. 0. 0. 0.] ground truth
JSD between network and ground truth upper path: 0.05912574287992064

The lower path produces:
network: [1. 0. 0. 0.] <-> [1. 0. 0. 0.] ground truth
JSD between network and ground truth upper path: nan

The abstraction path produces:
network: [0.998 0.001 0.001] <-> [0. 1. 0.] ground truth
JSD between network and ground truth upper path: 0.8299494371240532

Network JSD:
JSD between upper and lower path in the network: 0.0591131416098079

Ground truth JSD:
JSD between upper and lower path in the ground truth: nan

-------------------------
-------------------------

Given I perform the intervention A=[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]

The upper path produces:
network: [0.99  0.002 0.003 0.006] <-> [1. 0. 0. 0.] ground truth
JSD between network and ground truth upper path: 0.05912574287992064

T

-------------------------

Given I perform the intervention A=[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]

The upper path produces:
network: [0.99  0.002 0.003 0.006] <-> [1. 0. 0. 0.] ground truth
JSD between network and ground truth upper path: 0.05912574287992064

The lower path produces:
network: [1. 0. 0. 0.] <-> [1. 0. 0. 0.] ground truth
JSD between network and ground truth upper path: nan

The abstraction path produces:
network: [0.966 0.032 0.002] <-> [0. 1. 0.] ground truth
JSD between network and ground truth upper path: 0.7888984504761236

Network JSD:
JSD between upper and lower path in the network: 0.0591131416098079

Ground truth JSD:
JSD between upper and lower path in the ground truth: nan

-------------------------
-------------------------

Given I perform the intervention A=[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]

The upper path produces:
network: [0.99  0.003 0.003 0.004] <-> [0. 0. 0. 1.] ground truth
JSD between network and ground truth upper path: 0.8245336046793458

Th

In [255]:
print(f"Low Abstraction Accuracy: {abs_low_acc/n_test}")
np.set_printoptions(precision=3, suppress=True)
# Construct alpha weight_ from network:
res = np.zeros((3,12))
res1 = np.zeros((3,12))
for i in range(12):
    tmp = np.array([0,0,0,0,0,0,0,0,0,0,0,0])
    tmp[i] = 1
    test_mech_low = torch.from_numpy(np.dot(tmp, mech_low.T))
    data = torch.from_numpy(tmp).float()
    abs_low, abs_mech_high, mech_low_abs = model(data, test_mech_low.float(), test_mech_high)
    idx = np.argmax(abs_low.detach().numpy())
    res[idx,i] = 1
    res1[:,i] = abs_low.detach().numpy()
    upper_path = np.dot(tmp,mech_low.T)
    
print("Alpha weight_ from network:")
print(res)
print(res1)
print("Alpha weight_ from ground truth:")
print(alphas['Weight_'])

# Construct alpha range_ from network:
res_r = np.zeros((4,4))
res1_r = np.zeros((4,4))
for i in range(4):
    tmp = np.array([0,0,0,0])
    tmp[i] = 1
    alpha_range = list(model.parameters())[2].detach().numpy()
    out = np.dot(tmp,alpha_range)
    out = nn.functional.softmax(torch.from_numpy(out))
    idx = np.argmax(out)
    res_r[idx,i] = 1
    res1_r[:,i] = out
    
    
print("Alpha range_ from network:")
print(res_r)
print(res1_r)
print("Alpha range_ from ground truth:")
print(alphas['Range_'])

Low Abstraction Accuracy: 0.42
Alpha weight_ from network:
[[1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1.]
 [0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0.998 0.965 0.343 0.966 0.993 0.182 0.984 0.995 0.988 0.778 0.98  0.869]
 [0.001 0.031 0.58  0.032 0.006 0.816 0.006 0.003 0.003 0.195 0.019 0.13 ]
 [0.001 0.005 0.077 0.002 0.001 0.003 0.01  0.002 0.009 0.027 0.002 0.001]]
Alpha weight_ from ground truth:
[[0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1.]
 [1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.]]
Alpha range_ from network:
[[1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 1. 1. 1.]]
[[0.365 0.148 0.216 0.258]
 [0.278 0.155 0.212 0.144]
 [0.24  0.156 0.104 0.156]
 [0.117 0.541 0.468 0.442]]
Alpha range_ from ground truth:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 1. 1. 0.]]


/var/folders/n0/yr8s1fh94295209wh5ynccp40000gn/T/ipykernel_2764/91320153.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  abs_low = self.absS(out)
/var/folders/n0/yr8s1fh94295209wh5ynccp40000gn/T/ipykernel_2764/91320153.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  mech_low_abs = self.absC(out)
/var/folders/n0/yr8s1fh94295209wh5ynccp40000gn/T/ipykernel_2764/2883709353.py:31: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = nn.functional.softmax(torch.from_numpy(out))
